In [11]:
import numpy as np

sims = 50
nu = 0
T = int(1e6)
mu1 = 1
M = 10
thresholds = [1e3,2e3,3e3,4e3,5e3]

In [27]:
def xumei(data, threshold, lower):
    W = np.zeros(data.shape[0])
    statistic = max(W)
    stream = -1
    t = -1
    while statistic < threshold:
        stream = (stream + 1)%M
        m_t = 0
        sum_t = 0
        while W[stream]>=0 and W[stream]<threshold:
            t = t+1
            x_t = data[stream,t]
            if m_t == 0:
                mean_estimate = lower
            else:
                mean_estimate = max((sum_t/m_t,lower))
            W[stream] = max((W[stream],0))+mean_estimate*x_t-(mean_estimate**2)/2
            for i in range(data.shape[0]):
                if i != stream:
                    W[i] = max((W[i],0))
            m_t = m_t + 1
            sum_t = sum_t + x_t
        statistic = max(W)
    return t

In [114]:
import numpy as np
import focus_implementation as focus

data = focus.generate_data(M, T, int(1e5), mu1)
data[[0, -1], :] = data[[-1, 0], :]
detection_delay = 0
sims = 50
sim = 0
while sim<sims:
    delay = xumei(data,1000,2)-1e5
    if delay > 0:
        sim = sim+1
        detection_delay = detection_delay + delay
print(detection_delay/50)

75883.0


In [ ]:
import numpy as np
import focus_implementation as focus

edd_means = np.zeros((len(nus), len(thresholds)))

for idx, nu in enumerate(nus):
    for idx2, threshold in enumerate(thresholds):
        delays = []
        sim = 0
        while sim<sims:
            np.random.seed()
            data = focus.generate_data(M, T, nu, mu1)
            detection_time, _ = focus.focus_decay(data, threshold)
            if detection_time > nu:
                sim = sim+1
                delays.append(detection_time - nu)
        # This is just printing off results
        print(f"changepoint: {nu}")
        print(f"threshold: {threshold}")
        lai_bound = threshold * 2 / (mu1 ** 2)
        print(f"asymptotic bound: {lai_bound}")
        edd_estimate = sum(delays) / sims
        print(f"edd: {edd_estimate}")
        print(f"asymptotic bound ratio: {edd_estimate / lai_bound}")
        edd_means[idx][idx2] = edd_estimate
        print("--------------------------------------------------")